# Sed log maker

Basic program to produce sedimentary logs. See example input file for required format.

Prerequisites:
- Numpy
- Pandas
- drawSvg

Most Python installs will include numpy and pandas by default, but probably not drawSvg. It can be obtained from PyPi as follows:

`$ pip3 install drawSvg`

Import modules for running program. If drawings fails to import, uncomment the syspath block and make sure your present working directory is the directory this repo is cloned to. Running this cell will almost definitely throw an error about Cairo not being installed, but unless you desperately have to export your file as a PNG, you can ignore this.

In [1]:
import numpy as np
import pandas as pd
import drawings as dr

## Input parameters

In [2]:
## Specify this stuff
filepath = 'testdata.csv' # Path to your spreadsheet
output = 'testoutput.svg' # Path to output file
export = True             # True to export file, False to not do that

## Don't change this stuff unless you really want to
src = pd.read_csv(filepath)      # Read spreadsheet as pd.DataFrame
src = src.fillna('NaN')          # Fill blank cells
elevations = dr.elevs(src.thickness)

In [3]:
## Definitely do specify this stuff
vscale = 500       # Vertical scale of log in terms of x:1
                   #(i.e., vscale of 500 prints final log where 500 mm rock corresponds to 1 mm log)
tick_interval = 20 # How many metres apart do you want your ticks to be?
label = src.labels # Column to use as labels for units
                   # Program also accepts 'facies' or 'number' if you just want labelling to show the facies or unit number
                   # or None if you don't want any labels at all.

## Probably don't bother changing this stuff
origin = 40            # X and Y distance from bottom corner of page to start log in pt
padding = 5            # Padding around the edge of the page in pt
column_spacing = 40    # Space between columns in pt
line_weight = 0.5      # Line weight in pt
strat = ''       # Labelling mode; 'polite' for labels that avoid writing on top of units, '' for labelling everything
manual_column_height = None # Specify a given column height
cols = None            # Specify a number of columns

The cells below specify the lookup lists that are used to make everything the right shape and size in your log. The default lists are as follows:
`grainsize_codes = np.array(["NaN","cl","si","vf","f","m","c","vc","gr","pebb","cobb","boul"])`

`grainsize_widths = np.array([7.50, 15.00, 22.50, 30.00, 33.75, 37.50, 41.25, 45.00, 52.50, 60.00, 67.50, 75.00])`

`facies_codes = np.array("inaccessible","cov",
                            "fcm","fcl","fcr","fcrc","fcrw","fsm","fsl","fsr","fsrc","fsrw",
                            "sm","sh","sp","st","sr","src","srw","gmm","gmmi","gcm","gcmi","gcp","gct","gch")`
                            
`facies_colors = np.array("#FFFFFF","#FFFFFF","#5A4854","#74576A","#8C627E","#A57093","#D998C1",
                            "#666154","#807969","#99917D","#B3A993","#CCC1A7","#E3AB4A","#DBB75C",
                            "#FDBB45","#FBB672","#FFCE6F","#EE9621","#F68C35","#99834F","#E5C376",
                            "#661714","#8D211D","#B22C26","#D93328","#EF4130")`
                            
The default canvas created by `dr.canvas` is portrait US letter. If you wish to create alternatives to better suit your application, you are referred to the `drawings.py` file and embedded help text (which I will be improving at some point). The pallette is also in the repo if you (like me, I hope) can't read hex codes.

If you're unsure, just leave default as true and create your input file accordingly.

In [4]:
default = True

In [5]:
if(default is True):
    gs_codes, gs_widths = dr.grainsize()
    fa_codes, fa_colors = dr.faciesList()
    canv = dr.canvas()
else:
    gs_codes = []
    gs_widths = []
    fa_codes = []
    fa_colors = []
    canvas_x = 1
    canvas_y = 1
    canv = dr.canvas(canvas_x, canvas_y)

## Generate the log

This is where the actual generation takes place. It's normally pretty snappy, but I haven't tested for more than about 500 units so your mileage may vary.

In [6]:
%%time

log = dr.drawLog(elevations, vscale,
             src.gs_base, src.gs_top, src.code,
             gs_codes, gs_widths, fa_codes, fa_colors, canv,
             orig = origin, pad = padding, colspc = column_spacing, lnwgt = line_weight,
             man_colheight = manual_column_height, columns = cols, ticks = tick_interval,
             labels = label, label_strat = strat, nachar = 'NaN')

CPU times: user 122 ms, sys: 92 µs, total: 122 ms
Wall time: 122 ms


In [7]:
if export is True:
    log.saveSvg(output)